In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


In [6]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [7]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 34950.539062, Training Accuracy= 0.13281
Iter 2560, Minibatch Loss= 16023.212891, Training Accuracy= 0.33594
Iter 3840, Minibatch Loss= 8867.554688, Training Accuracy= 0.51562
Iter 5120, Minibatch Loss= 3816.481689, Training Accuracy= 0.71094
Iter 6400, Minibatch Loss= 4115.771484, Training Accuracy= 0.76562
Iter 7680, Minibatch Loss= 7676.147949, Training Accuracy= 0.74219
Iter 8960, Minibatch Loss= 3575.384277, Training Accuracy= 0.79688
Iter 10240, Minibatch Loss= 3659.337646, Training Accuracy= 0.77344
Iter 11520, Minibatch Loss= 1674.292725, Training Accuracy= 0.89844
Iter 12800, Minibatch Loss= 2762.049316, Training Accuracy= 0.85156
Iter 14080, Minibatch Loss= 1203.178467, Training Accuracy= 0.89844
Iter 15360, Minibatch Loss= 1012.428284, Training Accuracy= 0.92188
Iter 16640, Minibatch Loss= 1847.598999, Training Accuracy= 0.88281
Iter 17920, Minibatch Loss= 978.580078, Training Accuracy= 0.91406
Iter 19200, Minibatch Loss= 1226.933838, Training Accu

Iter 157440, Minibatch Loss= 289.120300, Training Accuracy= 0.96094
Iter 158720, Minibatch Loss= 341.674927, Training Accuracy= 0.96094
Iter 160000, Minibatch Loss= 263.231903, Training Accuracy= 0.97656
Iter 161280, Minibatch Loss= 283.622009, Training Accuracy= 0.94531
Iter 162560, Minibatch Loss= 649.557739, Training Accuracy= 0.95312
Iter 163840, Minibatch Loss= 19.485168, Training Accuracy= 0.98438
Iter 165120, Minibatch Loss= 432.265228, Training Accuracy= 0.96875
Iter 166400, Minibatch Loss= 281.725647, Training Accuracy= 0.96094
Iter 167680, Minibatch Loss= 134.668488, Training Accuracy= 0.97656
Iter 168960, Minibatch Loss= 272.558838, Training Accuracy= 0.94531
Iter 170240, Minibatch Loss= 107.865326, Training Accuracy= 0.98438
Iter 171520, Minibatch Loss= 210.760498, Training Accuracy= 0.97656
Iter 172800, Minibatch Loss= 49.782326, Training Accuracy= 0.99219
Iter 174080, Minibatch Loss= 86.150879, Training Accuracy= 0.97656
Iter 175360, Minibatch Loss= 252.988953, Training A